In [ ]:
!pip install openai pydantic

In [ ]:
from openai import OpenAI
from google.colab import userdata

client = OpenAI(
    api_key=userdata.get("OPENROUTER_API_KEY"),
    base_url="https://openrouter.ai/api/v1",  # OpenRouter OpenAI-compatible base URL
)

In [ ]:
from typing import Dict, Any, List

import requests
import unicodedata


# retrieve data from notion

NOTION_TOKEN = userdata.get("NOTION_API")
PAGE_ID = "2ee0a735274f80b6a961ee81eed44b06"
NOTION_VERSION = "2025-09-03"

headers = {
    "Authorization": f"Bearer {NOTION_TOKEN}",
    "Notion-Version": NOTION_VERSION,
    "Content-Type": "application/json",
}

def notion_get(url: str, params: Dict[str, Any] | None = None) -> Dict[str, Any]:
    r = requests.get(url, headers=headers, params=params)
    r.raise_for_status()
    return r.json()

def list_block_children(block_id: str) -> List[Dict[str, Any]]:
    """Fetches all child blocks (paginated)."""
    results = []
    url = f"https://api.notion.com/v1/blocks/{block_id}/children"
    cursor = None
    while True:
        params = {"start_cursor": cursor} if cursor else None
        data = notion_get(url, params=params)
        results.extend(data.get("results", []))
        if not data.get("has_more"):
            break
        cursor = data.get("next_cursor")
    return results

def rich_text_to_plain(rt: List[Dict[str, Any]]) -> str:
    return "".join(x.get("plain_text", "") for x in (rt or []))

def block_to_text(block: Dict[str, Any]) -> str:
    t = block["type"]
    if t in ("paragraph", "heading_1", "heading_2", "heading_3", "bulleted_list_item", "numbered_list_item", "to_do", "quote", "callout"):
        obj = block[t]
        return rich_text_to_plain(obj.get("rich_text", []))
    if t == "code":
        obj = block[t]
        lang = obj.get("language", "")
        code = rich_text_to_plain(obj.get("rich_text", []))
        return f"```{lang}\n{code}\n```"
    if t == "divider":
        return "---"
    if t == "image":
        return block["image"].get("caption", [{}])[0].get("plain_text", "") or "[image]"
    return f"[{t}]"
def normalize(text: str)-> str:
    return unicodedata.normalize("NFKD", text)

def walk_blocks(block_id: str, indent: int = 0) -> List[str]:
    lines = []
    children = list_block_children(block_id)
    for b in children:
        prefix = "  " * indent
        text = block_to_text(b)
        if text.strip():
            lines.append(prefix + text)
        if b.get("has_children"):
            lines.extend(walk_blocks(b["id"], indent=indent + 1))
    return normalize("\n".join(lines))




lines = walk_blocks(PAGE_ID)







In [ ]:
from datetime import date
from pydantic import BaseModel, Field, ValidationError, field_validator, model_validator

class Status(BaseModel):
    status: str = Field(min_length=3, description="The post text to publish on Mastodon (<= 100 words).")
    date_posted: date = Field(description="Target date for posting (YYYY-MM-DD).")
    content: str = Field(description="Short internal rationale or angle for the post (not posted).")

class StatusList(BaseModel):
    posts: List[Status] = Field(min_length=1, description="List of generated posts.")


num_posts = 5
text_instructions = f"make {num_posts} post markting my company under 100 words using the following information: " + lines + "make sure that each post can be followed by an image."
posts = client.responses.parse(
      model="nvidia/nemotron-3-nano-30b-a3b:free",
      input= text_instructions,
      text_format=StatusList,
  )

In [ ]:
parsed = (
    getattr(posts, "output_parsed", None)
    or getattr(posts, "parsed", None)
    or posts
)

In [ ]:
!pip -q install replicate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 4.1 MB/s eta 0:00:00


In [ ]:
os.environ["REPLICATE_API_TOKEN"] = userdata.get("Replicate_key")

import os
import replicate
import requests

TRIGGER_WORD = 'TEMP'




replicate_url = "https://replicate.com/cesiam/memories_with_lucas"
replicate_model = "cesiam/memories_with_lucas"

# def get_latest_version_id(model_name: str) -> str:
#     """
#     Fetches a model and returns a version id.
#     Assumes versions.list() is available; typically returns newest-first.
#     """
#     model = replicate.models.get(model_name)
#     versions = list(model.versions.list())
#     if not versions:
#         raise RuntimeError(f"No versions found for model: {model_name}")
#     return versions[0].id  # usually latest

# version_id = get_latest_version_id(replicate_model)
# model_version = f"{replicate_model}:{version_id}"
model_version = 'cesiam/memories_with_lucas:f3a6bd0b3b3ec58a76807201e533e161112b04096f63cfdf50f9fd96915b608b'
print("Using:", model_version)

def generate_images_for_posts(posts: List[Status], out_dir: str = "/content/generated_images"):
    os.makedirs(out_dir, exist_ok=True)
    image_paths = []

    for i, p in enumerate(posts, start=1):
        # Assumes the model takes {"prompt": "..."}; adjust if your model uses a different key.
        print(type(p.content), type(TRIGGER_WORD))
        image_insturctions = f"generate an image based on the following text, making sure that {TRIGGER_WORD}'s face appears in the generated image: {p.content}"
        output = replicate.run(model_version, input={"prompt": image_insturctions})

        # Replicate often returns a list of FileOutput objects for images
        file_output = output[0] if isinstance(output, list) else output

        out_path = os.path.join(out_dir, f"post_{i}.png")
        with open(out_path, "wb") as f:
            f.write(file_output.read())

        image_paths.append(out_path)

    return image_paths



Using: cesiam/memories_with_lucas:f3a6bd0b3b3ec58a76807201e533e161112b04096f63cfdf50f9fd96915b608b


In [ ]:
import os
import requests
from typing import Optional, List

MASTODON_INSTANCE = "https://mastodon.social"
ACCESS_TOKEN = userdata.get("MASTODON_API")    # token with write:statuses + write:media
visibility = 'unlisted'
def upload_media(
    instance: str,
    token: str,
    image_path: str,
    alt_text: str,
) -> str:
    """
    Upload an image to Mastodon and return media_id.
    Uses /api/v2/media (preferred). Falls back to /api/v1/media if needed.
    """
    headers = {"Authorization": f"Bearer {token}"}

    def _try_upload(url: str) -> Optional[str]:
        with open(image_path, "rb") as f:
            files = {"file": f}
            data = {"description": alt_text}
            r = requests.post(url, headers=headers, files=files, data=data)
        if r.status_code == 404:
            return None
        r.raise_for_status()
        return r.json()["id"]

    media_id = _try_upload(f"{instance}/api/v2/media")
    if media_id is None:
        media_id = _try_upload(f"{instance}/api/v1/media")
        if media_id is None:
            raise RuntimeError("Media upload endpoint not found on this instance.")

    return media_id

def post_status_with_media(
    instance: str,
    token: str,
    status_text: str,
    media_id: str,
    visibility: str = "unlisted",
    spoiler_text: Optional[str] = None,
) -> str:
    """
    Post a status with a single media attachment. Returns the created status URL.
    """
    url = f"{instance}/api/v1/statuses"
    headers = {"Authorization": f"Bearer {token}"}

    payload = {
        "status": status_text,
        "visibility": visibility,   # public | unlisted | private | direct
        "media_ids[]": [media_id],  # form encoding expects media_ids[]
    }
    if spoiler_text:
        payload["spoiler_text"] = spoiler_text

    r = requests.post(url, headers=headers, data=payload)
    r.raise_for_status()
    return r.json().get("url", "")





In [ ]:
type(parsed)

__main__.StatusList

In [ ]:
HASHTAGS = " #marketing #AIGenerated"
VISIBILITY = "unlisted"


posted_urls: List[str] = []
image_paths = generate_images_for_posts(parsed.posts)
for post, img_path in zip(posts, image_paths):
    text = (post.status.strip() + HASHTAGS).strip()

    media_id = upload_media(
        instance=MASTODON_INSTANCE,
        token=ACCESS_TOKEN,
        image_path=img_path,
        alt_text=post.alt_text,
    )

    url = post_status_with_media(
        instance=MASTODON_INSTANCE,
        token=ACCESS_TOKEN,
        status_text=text,
        media_id=media_id,
        visibility=VISIBILITY,
        # spoiler_text="CW: AI-generated"  # optional
    )

    posted_urls.append(url)
    print("Posted:", url)

posted_urls


In [ ]:
# generate 5 mastordon posts with images generated from my fined model in replicate
